# Load Data

In [1]:
import json
from pathlib import Path
from operator import itemgetter
import itertools
import random

classify_path = Path('/mnt/data/factcheck/tweets/classify')
all_lines_path = Path(classify_path, 'data', 'all_lines.jsonl')

def load_jsonl(path):
    with open(path) as f:
        return [json.loads(line) for line in f]
    
all_lines = load_jsonl(all_lines_path)
random.shuffle(all_lines)
print('all lines loaded:', len(all_lines), 'items')
print(all_lines[0])

all lines loaded: 2500 items
{'id': 1247517397675724800, 'text': ' ideologizace epidemii, které byly a budou, je úsměvná. Gravitace není taky kapitalistická nebo socialistická', 'label': 0}


# Create Datasets and Tokenize Them

In [2]:
import json
import random
from collections import defaultdict
from datasets import Dataset
import copy
    
def get_dataset(data):
    reordered = {
        'text': [item['text'] for item in data],
        'label': [item['label'] for item in data],
        'idlink': [item['id'] for item in data],
        'idsmall': list(range(len(data)))
    }
    return Dataset.from_dict(reordered)

data = get_dataset(all_lines)
texts = copy.deepcopy(data)
print(data)

Dataset({
    features: ['text', 'label', 'idlink', 'idsmall'],
    num_rows: 2500
})


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import DataCollatorWithPadding
import torch

classify_path = Path('/mnt/data/factcheck/tweets/classify')
tokenizer_path = Path(classify_path, 'models', 'xlm-roberta-tokenizer')
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

def tokenize_and_format_dataset(dataset):
    tokenize_fun = lambda example: tokenizer(example['text'], truncation=True)
    dataset = dataset.map(tokenize_fun, batched=True)
    dataset = dataset.rename_column('label', 'labels')
    dataset = dataset.remove_columns(['text'])
    dataset.set_format('torch')
    return dataset

tokenized = tokenize_and_format_dataset(data)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [4]:
import datasets
from datasets import DatasetDict
import random
from torch.utils.data import DataLoader

def shuffle_split_tokenized(tok_data, texts, test_frac: float, val_frac: float):
    assert len(tok_data)==len(texts)
    indices = list(range(len(tok_data)))
    random.shuffle(indices)
    limit1 = int(len(tok_data) * (1 - test_frac - val_frac))
    limit2 = int(len(tok_data) * (1 - val_frac))
    dataset = DatasetDict({
        'train': tok_data.select(indices[:limit1]),
        'test': tok_data.select(indices[limit1:limit2]),
        'valid': tok_data.select(indices[limit2:])
    })
    out_texts = {
        'train': [texts[index] for index in indices[:limit1]],
        'test': [texts[index] for index in indices[limit1:limit2]],
        'valid': [texts[index] for index in indices[limit2:]]
    }
    return dataset, out_texts

tokenized_splits, text_splits = shuffle_split_tokenized(tokenized, texts, 0.2, 0.2)
print(tokenized_splits)
print([(k,len(v)) for k, v in text_splits.items()])

# tk = DataLoader(tokenized_splits, batch_size=20, shuffle=False)
# dl = DataLoader(text_splits['train'], batch_size=10, shuffle=False)
# for b in dl:
#     print(b)
#     assert False

# for i, test in enumerate(tokenized_splits['valid']):
#     if test['labels'] == 1:
#         print(text_splits['valid'][i])

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idlink', 'idsmall', 'input_ids', 'labels'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['attention_mask', 'idlink', 'idsmall', 'input_ids', 'labels'],
        num_rows: 500
    })
    valid: Dataset({
        features: ['attention_mask', 'idlink', 'idsmall', 'input_ids', 'labels'],
        num_rows: 500
    })
})
[('train', 1500), ('test', 500), ('valid', 500)]


### Tests

In [5]:
tokenized_splits, text_splits = shuffle_split_tokenized(tokenized, texts, 0.2, 0.2)
for partition in 'train', 'test', 'valid':
    for a, b in zip(tokenized_splits[partition], text_splits[partition]):
        assert a['idsmall'] == b['idsmall']
print('test suceeded')

test suceeded


In [6]:
from torch.utils.data import DataLoader

batch_size = 256

# print(text_splits['train']['idsmall'].dtype, text_splits['train']['idsmall'])

for partition in 'train', 'test', 'valid':

    tens_data, text_data = tokenized_splits[partition], text_splits[partition]
    dl_tens = DataLoader(tens_data, batch_size=batch_size, collate_fn=data_collator, shuffle=False)
    dl_text = DataLoader(text_data, batch_size=batch_size, shuffle=False)

    for aa, bb in zip(dl_tens, dl_text):
        a, b = aa['idsmall'], bb['idsmall']
        assert (a == b).all()
        assert len(a) == len(b)
        for i in range(len(a)):
            assert a[i] == b[i]
        for one, two in zip(a, b):
            assert one == two

def evaluate_test(tens_data, text_data, collate_fn, batch_size=256):
    assert len(tens_data) == len(text_data)
    # print(text_data['idsmall'].dtype, text_data['idsmall'])
    # assert (tens_data['idsmall']==text_data['idsmall']).all()
    
    dl_tens = DataLoader(tens_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)
    dl_text = DataLoader(text_data, batch_size=batch_size, shuffle=False)
    
    for aa, bb in zip(dl_tens, dl_text):
        a, b = aa['idsmall'], bb['idsmall']
        assert (a == b).all()
        assert len(a) == len(b)
        for i in range(len(a)):
            assert a[i] == b[i]
        for one, two in zip(a, b):
            assert one == two
            
evaluate_test(tokenized_splits['test'], text_splits['test'], data_collator)
print('test suceeded')

test suceeded


# XLM Roberta Model

In [7]:
from itertools import compress
from torch.utils.data import DataLoader


class Metrics:
    def __init__(self):
        self.tp = self.fp = self.tn = self.fn = 0
        
    def update(self, truth, predictions):
        self.tp += ((truth==predictions) & (truth==1)).sum().item()
        self.tn += ((truth==predictions) & (truth==0)).sum().item()
        self.fp += ((truth!=predictions) & (truth==0)).sum().item()
        self.fn += ((truth!=predictions) & (truth==1)).sum().item()
        
    def compute(self, numerator, denominator):
        if sum(denominator) == 0:
            return 0
        return sum(numerator) / sum(denominator)
    
    @property
    def accuracy(self):
        return self.compute((self.tp, self.tn), (self.tp, self.tn, self.fp, self.fn))
    
    @property
    def precision(self):
        return self.compute((self.tp,), (self.tp, self.fp))
    
    @property
    def recall(self):
        return self.compute((self.tp,), (self.tp, self.fn))
        
    @property
    def score(self):
        # return 3 * self.precision + 2 * self.recall + self.accuracy
        # return self.tp * self.precision + (self.tp+self.tn+self.fp+self.fn)*self.precision
        return 2 * self.precision - (1-self.recall)**2 + self.accuracy
        # return self.precision - (1-self.recall)**2
        # return 2 * self.precision + self.recall
        
    def __repr__(self):
        res = f'tp {self.tp:03}, fp {self.fp:03}, tn {self.tn:03}, fn {self.fn:03}'
        labels = [('acc', self.accuracy), ('prec', self.precision), ('rec', self.recall)]
        parts = [f'{name} {var:.3f}' for name, var in labels if var is not None]
        return res + ' (' + ', '.join(parts) + ')'
    
    def to_json_string(self):
        def truncate_num(num):
            return round(num, 3) if isinstance(num, float) else num
        variables = 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall'
        dictionary = {v: truncate_num(getattr(self, v)) for v in variables}
        return json.dumps(dictionary)

    
classify_path = Path('/mnt/data/factcheck/tweets/classify')
model_path = Path(classify_path, 'models', 'xlm-roberta-base')
outfile_path = Path(classify_path, 'fp.txt')


def evaluate(model, device, tens_data, text_data, collate_fn, batch_size=256):
    assert len(tens_data) == len(text_data)
    # print(text_data['idsmall'].dtype, text_data['idsmall'])
    # assert (tens_data['idsmall']==text_data['idsmall']).all()
    
    dl_tens = DataLoader(tens_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)
    dl_text = DataLoader(text_data, batch_size=batch_size, shuffle=False)
    
    # print('\n\n', file=outfile)
    model.eval()
    metrics = Metrics()
    for batch, texts in zip(dl_tens, dl_text):        
        batch_inp = {k: v.to(device) for k, v in batch.items() if k not in ('idlink', 'idsmall')}
        with torch.no_grad():
            outputs = model(**batch_inp)
        predictions = outputs.logits.argmax(dim=-1)
        truth = batch['labels'].to(device)
        metrics.update(truth, predictions)
        with open(outfile_path, 'at') as f:
            ids = ((predictions==1) & (truth==0)).nonzero()
            for i in ids:
                print(texts['text'][i], file=f)            
        
    return metrics

#### Download model and save it

In [8]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base")
# from pathlib import Path
# classify_path = Path('/mnt/data/factcheck/tweets/classify')
# model_path = Path(classify_path, 'models', 'xlm-roberta-base')
# model.save_pretrained(model_path)

#### Work with saved model

In [ ]:
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
import torch
import copy
from pathlib import Path

classify_path = Path('/mnt/data/factcheck/tweets/classify')
model_path = Path(classify_path, 'models', 'xlm-roberta-base')
# model_path = Path(classify_path, 'models', 'bestmodel00')

# size 16, multi 4, 32/4, interesting 64/7
def run(tokenized=tokenized, data_collator=data_collator, 
        num_epochs=30, div_max=1, batch_size=16, multiplier=5, save_model=None):
    
    if torch.cuda.is_available():
            torch.cuda.empty_cache()

    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)

    tokenized_splits, text_splits = shuffle_split_tokenized(tokenized, texts, 0.2, 0.2)
    dataloaders = {key: DataLoader(tokenized_splits[key], 
                                   batch_size=batch_size, collate_fn=data_collator)
                   for key in tokenized_splits.keys()}
    dl_texts = {key: DataLoader(text_splits[key], batch_size=batch_size)
                   for key in text_splits.keys()}
    
    optimizer = AdamW(model.parameters(), lr=1e-5)

    num_training_steps = num_epochs * len(dataloaders['train'])

    lr_scheduler = get_scheduler(  # works better here w/o warmup
        "constant_with_warmup",
        num_warmup_steps = int(num_training_steps * 0),
        optimizer=optimizer,
    )

    progress_bar = tqdm(range(num_training_steps))
    best_score = 0
    best_metrics = None

    div = 0
    for epoch in range(num_epochs):
        model.train()
        for batch in dataloaders['train']:
            div += 1
            batch_inp = {k: v.to(device) for k, v in batch.items() if k not in ('idlink', 'idsmall')}
            outputs = model(**batch_inp)
            
            logits = outputs.logits
            n = logits.size()[0]
            logsm = torch.nn.functional.log_softmax(logits, dim=1)
            labels = batch_inp['labels']
            gat = logsm[range(n), labels]
            loss = -(gat * (labels==0).float()).mean()*multiplier \
                   -(gat * (labels==1).float()).mean()
        
            loss.backward()
            if div % div_max == 0:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
            progress_bar.update(1)

        metrics = evaluate(model, device, tokenized_splits['test'], text_splits['test'],
                          data_collator)
        print(f'epoch {epoch:02d}, loss {loss:.4f}, {metrics!r}')

        if metrics.score > best_score:
            if save_model is not None:  
                path = Path(classify_path, 'models', f'bestmodel{save_model:02}')
                model.save_pretrained(path)
            best_score = metrics.score
            best_metrics = copy.deepcopy(metrics)
            valid_metrics = evaluate(model, device, tokenized_splits['valid'], text_splits['valid'],
                                    data_collator)
            print('validation metrics    ', valid_metrics)

    res_path = Path(classify_path, 'metrics', 'results.txt')
    with open(res_path, 'at') as f:
        print(best_metrics.to_json_string(), file=f)
        print(valid_metrics.to_json_string(), file=f)
        print(file=f)
        
for i in range(20):
    print(f'RUN {i}')
    run(save_model=i)

RUN 0


  0%|          | 0/1410 [00:00<?, ?it/s]

epoch 00, loss 0.6152, tp 000, fp 000, tn 451, fn 049 (acc 0.902, prec 0.000, rec 0.000)
epoch 01, loss 0.5249, tp 000, fp 000, tn 451, fn 049 (acc 0.902, prec 0.000, rec 0.000)
epoch 02, loss 0.4033, tp 000, fp 000, tn 451, fn 049 (acc 0.902, prec 0.000, rec 0.000)
epoch 03, loss 0.4472, tp 000, fp 000, tn 451, fn 049 (acc 0.902, prec 0.000, rec 0.000)
epoch 04, loss 0.3298, tp 000, fp 000, tn 451, fn 049 (acc 0.902, prec 0.000, rec 0.000)
epoch 05, loss 0.1695, tp 030, fp 018, tn 433, fn 019 (acc 0.926, prec 0.625, rec 0.612)
validation metrics     tp 021, fp 013, tn 440, fn 026 (acc 0.922, prec 0.618, rec 0.447)
epoch 06, loss 0.1275, tp 045, fp 077, tn 374, fn 004 (acc 0.838, prec 0.369, rec 0.918)
epoch 07, loss 0.0660, tp 041, fp 039, tn 412, fn 008 (acc 0.906, prec 0.512, rec 0.837)
epoch 08, loss 0.0886, tp 044, fp 080, tn 371, fn 005 (acc 0.830, prec 0.355, rec 0.898)
epoch 09, loss 0.0449, tp 044, fp 104, tn 347, fn 005 (acc 0.782, prec 0.297, rec 0.898)
epoch 10, loss 0.0979

  0%|          | 0/1410 [00:00<?, ?it/s]

epoch 00, loss 0.6124, tp 000, fp 000, tn 461, fn 039 (acc 0.922, prec 0.000, rec 0.000)
epoch 01, loss 0.5044, tp 000, fp 000, tn 461, fn 039 (acc 0.922, prec 0.000, rec 0.000)
epoch 02, loss 0.3893, tp 000, fp 000, tn 461, fn 039 (acc 0.922, prec 0.000, rec 0.000)
epoch 03, loss 0.1886, tp 000, fp 000, tn 461, fn 039 (acc 0.922, prec 0.000, rec 0.000)
epoch 04, loss 0.1161, tp 000, fp 000, tn 461, fn 039 (acc 0.922, prec 0.000, rec 0.000)
epoch 05, loss 0.2935, tp 004, fp 000, tn 461, fn 035 (acc 0.930, prec 1.000, rec 0.103)
validation metrics     tp 007, fp 000, tn 454, fn 039 (acc 0.922, prec 1.000, rec 0.152)
epoch 06, loss 0.0724, tp 024, fp 015, tn 446, fn 015 (acc 0.940, prec 0.615, rec 0.615)
epoch 07, loss 0.1431, tp 023, fp 007, tn 454, fn 016 (acc 0.954, prec 0.767, rec 0.590)
validation metrics     tp 024, fp 012, tn 442, fn 022 (acc 0.932, prec 0.667, rec 0.522)
epoch 08, loss 0.1565, tp 011, fp 003, tn 458, fn 028 (acc 0.938, prec 0.786, rec 0.282)
epoch 09, loss 0.0318

  0%|          | 0/1410 [00:00<?, ?it/s]

epoch 00, loss 0.8787, tp 000, fp 000, tn 457, fn 043 (acc 0.914, prec 0.000, rec 0.000)
epoch 01, loss 0.6057, tp 000, fp 000, tn 457, fn 043 (acc 0.914, prec 0.000, rec 0.000)
epoch 02, loss 0.4292, tp 000, fp 000, tn 457, fn 043 (acc 0.914, prec 0.000, rec 0.000)
epoch 03, loss 0.4883, tp 000, fp 000, tn 457, fn 043 (acc 0.914, prec 0.000, rec 0.000)
epoch 04, loss 0.4577, tp 000, fp 000, tn 457, fn 043 (acc 0.914, prec 0.000, rec 0.000)
epoch 05, loss 0.3520, tp 021, fp 023, tn 434, fn 022 (acc 0.910, prec 0.477, rec 0.488)
validation metrics     tp 024, fp 023, tn 416, fn 037 (acc 0.880, prec 0.511, rec 0.393)
epoch 06, loss 0.3129, tp 026, fp 021, tn 436, fn 017 (acc 0.924, prec 0.553, rec 0.605)
validation metrics     tp 029, fp 021, tn 418, fn 032 (acc 0.894, prec 0.580, rec 0.475)
epoch 07, loss 0.3547, tp 024, fp 012, tn 445, fn 019 (acc 0.938, prec 0.667, rec 0.558)
validation metrics     tp 024, fp 009, tn 430, fn 037 (acc 0.908, prec 0.727, rec 0.393)
epoch 08, loss 0.3170